# Taller: Espaciotiempo de Schwarzschild

En este notebook, exploraremos la solución de Schwarzschild a las Ecuaciones de Einstein de la Relatividad General.

Seguiremos la siguiente estructura:
1. Definiremos la métrica de Schwarzschild usando `sympy`.
2. Calcularemos los símbolos de Christoffel.
3. Obtendremos la ecuación geodésica en este espaciotiempo.
4. Exploraremos la librería especializada `EinsteinPy`.

Necesitaremos las siguientes librerías de Python:
- `sympy` para cálculos simbólicos.
- `scipy` y `numpy` para integración numérica.
- `matplotlib` para visualización.
- `einsteinpy` para cálculos más especializados en RG (al final).

-------------
## Preparación

### Cargar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy import symbols, Function, Matrix, simplify, sin, cos
from scipy.integrate import solve_ivp
sp.init_printing()

### Definir la métrica de Schwarzschild

In [ ]:
# Definir variables simbolicas para sympy. Esto nos permitira hacer manipulaciones al estilo 'Wolfram Mathematica'

# Las Coordenadas
t, r, theta, phi = symbols('t r theta phi')
coords = [t, r, theta, phi]

# El parametro de masa
M = symbols('M', real=True, positive=True)

# definir las componentes de la metrica de Schwarzschild
g = sp.zeros(4)
g[0,0] = -(1 - 2*M/r)
g[1,1] = 1/(1 - 2*M/r)
g[2,2] = r**2
g[3,3] = r**2 * sp.sin(theta)**2

# ahora imprimimos la metrica en su forma matricial
g


## Ejercicio 1: Calcular los símbolos de Christoffel

Utilizando la métrica de Schwarzschild definida anteriormente, calcular los símbolos de Christoffel:
$$
\Gamma^a_{bc} = \frac{1}{2} g^{ad}(\partial_b g_{dc} + \partial_c g_{db} - \partial_d g_{bc})
$$

Comenzar calculando la métrica inversa y las derivadas.

In [ ]:
g_inv = simplify(g.inv())

# Notar que tenemos sumatoria solo sobre el indice d
Gamma = [[[
    simplify(1/2 * sum(g_inv[a,d] * 
        (sp.diff(g[d,c], coords[b]) + sp.diff(g[d,b], coords[c]) - sp.diff(g[b,c], coords[d]))
        for d in range(4)))
    for c in range(4)] for b in range(4)] for a in range(4)]

# Mostrar las componentes
Gamma[1][0][0]  # Γ^r_{tt}

#### Preguntas:

Cuantas componentes de $\Gamma^a_{bc}$ son distintas de cero?

Cuantas componentes independientes tiene? Puede argumentarlo de forma teorica?

## Ejercicio 2: La ecuación geodésica

La ecuación geodésica es:

$$
\frac{d^2 x^a}{d\tau^2} + \Gamma^a_{bc} \frac{dx^b}{d\tau} \frac{dx^c}{d\tau} = 0
$$


Analizar el movimiento geodésico en el plano ecuatorial, i.e. $\theta = \pi/2$ (Recordar que la metrica tiene simetria esferica)

In [ ]:
# En el plano ecuatorial, θ = π/2, y entonces podemos utilizar sin(θ)=1
subs_dict = {theta: sp.pi/2, sp.sin(theta): 1, sp.cos(theta): 0}
Gamma_simplified = [[[
    Gamma[a][b][c].subs(subs_dict) for c in range(4)] for b in range(4)] for a in range(4)]

# Definir variables y funciones simbólicas x^a(τ)
tau = symbols('tau')
x = [Function(str(c))(tau) for c in coords]
dx = [sp.diff(xi, tau) for xi in x]
ddx = [sp.diff(dxi, tau) for dxi in dx]

# Ecuación geodésica
geodesic_eq = [
    simplify(ddx[a] + sum(Gamma_simplified[a][b][c].subs([(coords[i], x[i]) for i in range(4)]) * dx[b] * dx[c]
    for b in range(4) for c in range(4)))
    for a in range(4)]

# Mostrar las ecuaciones en pantalla
geodesic_eq[1]  # ecuación para r(τ)

#### Preguntas:

Matemáticamente, qué tipo de ecuaciones son estas?

Como sería posible resolverlas?

------
## Ejercicio 3: La librería `EinsteinPy`

En esta parte, explorar libremente los tutoriales disponibles en 

https://einsteinpy.org/

Luego los comentaremos